# Balanced BS Tree: AVL Tree

## Agenda

1. Motives
2. "Balanced" binary trees
3. Essential mechanic: rotation
4. Out-of-balance scenarios & rotation recipes
5. Generalized AVL rebalancing (insertion)
6. Rebalancing on removal

## 1. Motives

In [2]:
class BSTree:
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right
            
    def __init__(self):
        self.size = 0
        self.root = None
    
    def add(self, val):
        assert(val not in self)
        def add_rec(node):
            if not node:
                return BSTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
                return node
            else:
                node.right = add_rec(node.right)
                return node
        self.root = add_rec(self.root)
        self.size += 1
        
    def __contains__(self, val):
        def contains_rec(node):
            if not node:
                return False
            elif val < node.val:
                return contains_rec(node.left)
            elif val > node.val:
                return contains_rec(node.right)
            else:
                return True
        return contains_rec(self.root)
    
    def __len__(self):
        return self.size
    
    def __delitem__(self, val):
        assert(val in self)
        def delitem_rec(node):
            if val < node.val:
                node.left = delitem_rec(node.left)
                return node
            elif val > node.val:
                node.right = delitem_rec(node.right)
                return node
            else:
                if not node.left and not node.right:
                    return None
                elif node.left and not node.right:
                    return node.left
                elif node.right and not node.left:
                    return node.right
                else:
                    # remove the largest value from the left subtree as a replacement
                    # for the root value of this tree
                    t = node.left
                    if not t.right:
                        node.val = t.val
                        node.left = t.left
                    else:
                        n = t
                        while n.right.right:
                            n = n.right
                        t = n.right
                        n.right = t.left
                        node.val = t.val
                    return node
                        
        self.root = delitem_rec(self.root)
        self.size -= 1
        
    def pprint(self, width=64):
        """Attempts to pretty-print this tree's contents."""
        height = self.height()
        nodes  = [(self.root, 0)]
        prev_level = 0
        repr_str = ''
        while nodes:
            n,level = nodes.pop(0)
            if prev_level != level:
                prev_level = level
                repr_str += '\n'
            if not n:
                if level < height-1:
                    nodes.extend([(None, level+1), (None, level+1)])
                repr_str += '{val:^{width}}'.format(val='-', width=width//2**level)
            elif n:
                if n.left or level < height-1:
                    nodes.append((n.left, level+1))
                if n.right or level < height-1:
                    nodes.append((n.right, level+1))
                repr_str += '{val:^{width}}'.format(val=n.val, width=width//2**level)
        print(repr_str)
    
    def height(self):
        """Returns the height of the longest branch of the tree."""
        def height_rec(t):
            if not t:
                return 0
            else:
                return max(1+height_rec(t.left), 1+height_rec(t.right))
        return height_rec(self.root)

In [3]:
t = BSTree()
for x in range(6):
    t.add(x)
t.pprint()

                               0                                
               -                               1                
       -               -               -               2        
   -       -       -       -       -       -       -       3    
 -   -   -   -   -   -   -   -   -   -   -   -   -   -   -   4  
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 5 


In [4]:
import sys
sys.setrecursionlimit(100)

t = BSTree()
for x in range(100):
    t.add(x)

RecursionError: maximum recursion depth exceeded

## 2. "Balanced" binary trees

There are different metrics we can use to determine whether a binary tree is balanced --- for example, we could consider the *number* of nodes on either side of a given node, the *height* of the subtrees on either side of a given node, or even the specific *configuration* of nodes in subtrees on either side of a given node (perhaps we only want to permit full or complete trees).

For our purposes, we are going to define a balanced binary tree based on height. Specifically, an AVL tree (a particular type of balanced binary tree), is one where the *AVL property* holds for every node in the tree. The AVL property states that the heights of the left and right subtrees of a given node can differ by *no more than one*.

An AVL tree is also *self-balancing*. I.e., whenever the AVL property may be violated by an insertion/deletion, it must be immediately fixed --- the AVL property must always hold for all nodes in the tree after completing an insertion/deletion operation. 

How do we do this?

## 3. Essential mechanic: rotation

In [5]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right

        def rotate_right(self):
            p = self
            c = p.left
            p.left, p.right, c.left, c.right = c.left, c, c.right, p.right
            p.val, c.val = c.val, p.val
            
            
    def add(self, val):
        assert(val not in self)
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
                return node
            else:
                node.right = add_rec(node.right)
                return node
        self.root = add_rec(self.root)
        self.size += 1

In [6]:
t = AVLTree()
for x in range(6, 0, -1):
    t.add(x)
t.pprint()

                               6                                
               5                               -                
       4               -               -               -        
   3       -       -       -       -       -       -       -    
 2   -   -   -   -   -   -   -   -   -   -   -   -   -   -   -  
1 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 


In [7]:
t.root.rotate_right()
t.pprint()

                               5                                
               4                               6                
       3               -               -               -        
   2       -       -       -       -       -       -       -    
 1   -   -   -   -   -   -   -   -   -   -   -   -   -   -   -  


In [8]:
t.root.rotate_right()
t.pprint()

                               4                                
               3                               5                
       2               -               -               6        
   1       -       -       -       -       -       -       -    


In [9]:
t.root.left.rotate_right()
t.pprint()

                               4                                
               2                               5                
       1               3               -               6        


In [45]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right
            
        def rotate_right(self):
            p = self
            c = self.left
            p.val, c.val = c.val, p.val
            p.left, p.right, c.left, c.right = c.left, c, c.right, p.right
        
        @staticmethod
        def height(n):
            if not n:
                return 0
            else:
                return max(1+AVLTree.Node.height(n.left), 1+AVLTree.Node.height(n.right))
                                    
    def add(self, val): #O(log N * log N)
        assert(val not in self)
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
            else:
                node.right = add_rec(node.right)
            # detect and fix imbalance in `node` before returning
            if AVLTree.Node.height(node.left) > AVLTree.Node.height(node.right) + 1:
                print('left side imbalanced detected; rotating')
                node.rotate_right()  
        
            return node
        
        self.root = add_rec(self.root)
        self.size += 1

In [11]:
val = 50
t = AVLTree()

In [12]:
# (evaluate multiple times with ctrl-enter)
t.add(val)
val -= 1
t.pprint()

                               50                               


In [13]:
val = 0
t = AVLTree()

In [14]:
# (evaluate multiple times with ctrl-enter)
t.add(val)
val += 1
t.pprint()

                               0                                


## 4. "Out-of-balance" scenarios & rotation recipes

In [ ]:
# "left-left" scenario
t = BSTree()
for x in [3, 2, 1]:
    t.add(x)
t.pprint()

In [ ]:
# "left-right" scenario
t = BSTree()
for x in [3, 1, 2]:
    t.add(x)
t.pprint()

In [ ]:
# "right-right" scenario
t = BSTree()
for x in [1, 2, 3]:
    t.add(x)
t.pprint()

In [ ]:
# "right-left" scenario
t = BSTree()
for x in [1, 3, 2]:
    t.add(x)
t.pprint()

## 5. Generalized AVL rebalancing (insertion)

In [15]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right

        def rotate_right(self):
            p = self
            c = self.left
            p.val, c.val = c.val, p.val
            p.left, p.right, c.left, c.right = c.left, c, c.right, p.right
            
        def rotate_left(self):
            pass
        
        @staticmethod
        def height(n):
            if not n:
                return 0
            else:
                return max(1+AVLTree.Node.height(n.left), 1+AVLTree.Node.height(n.right))
    
    @staticmethod
    def rebalance(t):
        if AVLTree.Node.height(t.left) > AVLTree.Node.height(t.right):
            if AVLTree.Node.height(t.left.left) >= AVLTree.Node.height(t.left.right):
                # left-left
                print('left-left imbalance detected')
                t.rotate_right
            else:
                # left-right
                print('left-right imbalance detected')
                t.left.rotate_left()
                t.rotate_right
        else:
            # right branch imbalance tests needed
            pass
            
    def add(self, val):
        assert(val not in self)
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
            else:
                node.right = add_rec(node.right)
            if abs(AVLTree.Node.height(node.left) - AVLTree.Node.height(node.right)) > 1:
                AVLTree.rebalance(node)
            return node
        self.root = add_rec(self.root)
        self.size += 1

In [16]:
t = AVLTree()
for x in [10, 5, 1]:
    t.add(x)
t.pprint()

left-left imbalance detected
                               10                               
               5                               -                
       1               -               -               -        


In [17]:
# broken!
t = AVLTree()
for x in [10, 5, 1, 2, 3]:
    t.add(x)
t.pprint()

left-left imbalance detected
left-right imbalance detected
left-left imbalance detected
left-right imbalance detected
left-left imbalance detected
                               10                               
               5                               -                
       1               -               -               -        
   -       2       -       -       -       -       -       -    
 -   -   -   3   -   -   -   -   -   -   -   -   -   -   -   -  


## 5. Rebalancing on removal

In [39]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right

        def rotate_right(self):
            n = self.left
            self.val, n.val = n.val, self.val
            self.left, n.left, self.right, n.right = n.left, n.right, n, self.right
        
        def rotate_left(self):
            pass
            
        @staticmethod
        def height(n):
            if not n:
                return 0
            else:
                return max(1+AVLTree.Node.height(n.left), 1+AVLTree.Node.height(n.right))
    
    @staticmethod
    def rebalance(t):
        if AVLTree.Node.height(t.left) > AVLTree.Node.height(t.right):
            if AVLTree.Node.height(t.left.left) >= AVLTree.Node.height(t.left.right):
                # left-left
                print('left-left imbalance detected')
                t.rotate_right()
            else:
                # left-right
                print('left-right imbalance detected')
                t.left.rotate_left()
                t.rotate_right()
        else:
            pass
            
    def add(self, val):
        assert(val not in self)
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
            else:
                node.right = add_rec(node.right)
            if abs(AVLTree.Node.height(node.left) - AVLTree.Node.height(node.right)) >= 2:
                AVLTree.rebalance(node)
            return node
        self.root = add_rec(self.root)
        self.size += 1
        
    def __delitem__(self, val):
        assert(val in self)
        def delitem_rec(node):
            if val < node.val:
                node.left = delitem_rec(node.left)
            elif val > node.val:
                node.right = delitem_rec(node.right)
            else:
                if not node.left and not node.right:
                    return None
                elif node.left and not node.right:
                    return node.left
                elif node.right and not node.left:
                    return node.right
                else:
                    # remove the largest value from the left subtree (t) as a replacement
                    # for the root value of this tree
                    to_del = node.left
                    if not to_del.right:
                        node.left = to_del.left                        
                    else:
                        par = to_del
                        to_del = par.right
                        to_fix = [par]
                        while to_del.right:
                            par = par.right
                            to_fix.append(par)
                            to_del = to_del.right
                        # to_del refers to the right-most node, n par to its parent
                        par.right = to_del.left
                    node.val = to_del.val
                    
                    for n in to_fix[::-1]:
                        print('Checking', n.val)
                        if abs(AVLTree.Node.height(n.left) - AVLTree.Node.height(n.right)) > 1:
                            AVLTree.rebalance(n)
                            
            # detect and fix imbalance
            if abs(AVLTree.Node.height(node.left) - AVLTree.Node.height(node.right)) > 1:
                AVLTree.rebalance(node)
                
            return node
                        
        self.root = delitem_rec(self.root)
        self.size -= 1

In [40]:
t = AVLTree()
for x in [10, 5, 15, 2]:
    t.add(x)
t.pprint()

                               10                               
               5                               15               
       2               -               -               -        


In [41]:
del t[15]
t.pprint()

left-left imbalance detected
                               5                                
               2                               10               


In [42]:
t = AVLTree()
for x in range(31, 0, -1):
    t.add(x)
t.pprint()

left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
left-left imbalance detected
                               16                               
               8                               24               
       4               12              20              28       
   2       6       10      14      18      22      

In [43]:
del t[15]
del t[14]
t.pprint()

                               16                               
               8                               24               
       4               12              20              28       
   2       6       10      13      18      22      26      30   
 1   3   5   7   9   11  -   -   17  19  21  23  25  27  29  31 


In [44]:
del t[16]
t.pprint()

Checking 12
left-left imbalance detected
Checking 8
                               13                               
               8                               24               
       4               10              20              28       
   2       6       9       12      18      22      26      30   
 1   3   5   7   -   -   11  -   17  19  21  23  25  27  29  31 
